# Data Wrangling and Visualization

In this tutorial, we will be exploring the National Oceanic and Atmospheric Administration (NOAA) climate data set. Using plotly and managing database through sqlite3, we will be creating several interesting and interactive data graphics.

## Create a Database

First, let's create a database with three tables, `temperatures`, `stations`, and `countries`. They respectively record data on temperatures recorded at climate stations, information about climate stations, and information about countries.

As always, we will import necessary libraries at the start.

In [5]:
# import necessary libraries
import pandas as pd
import numpy as np
import sqlite3
import inspect
import plotly.express as px
import plotly.io as pio
pio.renderers.default="iframe"

First, let's create a database connection and name it `climate.db`.

In [7]:
# create database connection
conn = sqlite3.connect("climate.db")

Let's first download the files.

In [9]:
# load the temperatures data

import os
# create folder named "datafiles" if it does not exist
if not os.path.exists("datafiles"): 
    os.mkdir("datafiles")

# download the files
import urllib.request
intervals = [f"{i}-{i+9}" for i in range(1901, 2020, 10)]
for interval in intervals:
    url = f"https://raw.githubusercontent.com/PIC16B-ucla/25W/main/datasets/noaa-ghcn/decades/{interval}.csv"
    urllib.request.urlretrieve(url, f"datafiles/{interval}.csv")

To format our dataframe from the wide format to the long format, we will write a function `prepare_df()` to help us prepare the dataframe for easier analysis. We will also clean the data by removing rows with temperature of NaN.

In [11]:
def prepare_df(df):
    """
    prepares a piece of wide format dataframe into a long format data frame
    """
    # melt to the long format table
    df = df.melt(
        id_vars = ["ID", "Year"],
        value_vars = [f"VALUE{i}" for i in range(1, 13)],
        var_name = "Month",
        value_name = "Temp"
    )

    # cleaning month and temp
    df["Month"] = df["Month"].str[5:].astype(int)
    df["Temp"]  = df["Temp"] / 100
    df = df[~np.isnan(df["Temp"])] # remove rows with temperature of NaN 

    return df

Next, let's consolidate all the temperature data from 1901 to 2020 to a single table, which will then be added to the climate database.

In [13]:
# add the temperatures table to the database
intervals = [f"{i}-{i+9}" for i in range(1901, 2020, 10)]
for i, interval in enumerate(intervals):
    filepath = f"datafiles/{interval}.csv"
    df = pd.read_csv(filepath)
    df = prepare_df(df)
    df.to_sql("temperatures", conn, if_exists = "replace" if i == 0 else "append", index = False)

The data for stations and countries, respectively stored in a single csv file, will then be loaded and added as tables to the database.

In [15]:
# add the stations table to the database
stations = pd.read_csv("https://raw.githubusercontent.com/PIC16B-ucla/25W/refs/heads/main/datasets/noaa-ghcn/station-metadata.csv")
stations.to_sql("stations", conn, if_exists = "replace", index=False)

# add the countries table to the database
countries = pd.read_csv("https://raw.githubusercontent.com/mysociety/gaze/master/data/fips-10-4-to-iso-country-codes.csv")
countries.to_sql("countries", conn, if_exists = "replace", index=False)

279

Now let's verify whether we have successfully added the three tables to our database. We can also check the names of their columns.

In [17]:
cursor = conn.cursor()
cursor.execute("SELECT sql FROM sqlite_master WHERE type='table';")

for result in cursor.fetchall():
    print(result[0])

CREATE TABLE "temperatures" (
"ID" TEXT,
  "Year" INTEGER,
  "Month" INTEGER,
  "Temp" REAL
)
CREATE TABLE "stations" (
"ID" TEXT,
  "LATITUDE" REAL,
  "LONGITUDE" REAL,
  "STNELEV" REAL,
  "NAME" TEXT
)
CREATE TABLE "countries" (
"FIPS 10-4" TEXT,
  "ISO 3166" TEXT,
  "Name" TEXT
)


Remember to close the database connection after finishing constructing the database!

In [19]:
conn.close()

## Write a Query Function

Now, let's write a query function called `query_climate_database` that makes our workflow easier. This function is designed to help us retrieve information from the climate database all at once, such that we do not need to work on the three tables separately.

The function will take in several input parameters that specifies the country and time range for data extraction from the climate database, and will output a dataframe displaying comprehensive information including stations in this country, their location, time for temperature records, and the recorded temperature. Let's take a close look at the function:

In [22]:
from climate_database import query_climate_database
print(inspect.getsource(query_climate_database))

def query_climate_database(db_file, country, year_begin, year_end, month):
    """
    input:
    db_file: the file name for the database
    country: a string giving the name of a country for which data should be returned.
    year_begin and year_end: two integers giving the earliest and latest years for which should be returned (inclusive).
    month: an integer giving the month of the year for which should be returned.
    
    output (dataframe):
    NAME: The station name.
    LATITUDE: The latitude of the station.
    LONGITUDE: The longitude of the station.
    Country: The name of the country in which the station is located.
    Year: The year in which the reading was taken.
    Month: The month in which the reading was taken.
    Temp: The average temperature at the specified station during the specified year and month.
    """

    # connect to database
    conn = sqlite3.connect(db_file)

    # write sql command using f-strings
    command = f"""
    SELECT S.NAME, S.LATITUD

Now let's use this function to retrieve some data from the climate database. Looking at India's temperature data in January from 1980 to 2020, we get:

In [24]:
query_climate_database(db_file = "climate.db",
                       country = "India", 
                       year_begin = 1980, 
                       year_end = 2020,
                       month = 1)

,NAME,LATITUDE,LONGITUDE,Country,Year,Month,Temp
0,AGARTALA,23.883,91.250,India,1980,1,18.21
1,AGARTALA,23.883,91.250,India,1981,1,18.25
2,AGARTALA,23.883,91.250,India,1982,1,19.31
3,AGARTALA,23.883,91.250,India,1985,1,19.25
4,AGARTALA,23.883,91.250,India,1988,1,19.54
...,...,...,...,...,...,...,...
3147,VISHAKHAPATNAM,17.717,83.233,India,2016,1,25.09
3148,VISHAKHAPATNAM,17.717,83.233,India,2017,1,23.90
3149,VISHAKHAPATNAM,17.717,83.233,India,2018,1,22.65
3150,VISHAKHAPATNAM,17.717,83.233,India,2019,1,22.20


## Write a Geographic Scatter Function for Yearly Temperature Increases

Now that we have played around with our database, let's explore it and create some interesting visualizations.

Our guiding question will be: How does the average yearly change in temperature vary within a given country?

In order to create an interesting visualization to answer this question, let us create a geographic scatterplot in plotly express. Each point on the plot will represent the location of a station, and the color of the point reflects an estimate of the yearly change in temperature during the specific month and time period at that station.

In [28]:
from climate_database import temperature_coefficient_plot
print(inspect.getsource(temperature_coefficient_plot))

def temperature_coefficient_plot(db_file, country, year_begin, year_end, month, min_obs, **kwargs):
    """
    inputs:
    the first five are the same
    min_obs: the minimum required number of years of data for any given station
    **kwargs: additional keyword arguments passed to px.scatter_mapbox()
    """
    
    # query the database using the function we wrote earlier
    df = query_climate_database(db_file, country, year_begin, year_end, month)

    # filter to have stations which has more than or equal to min_obs
    df = df.groupby(["NAME", "LATITUDE", "LONGITUDE"]).filter(lambda group: len(group) >= min_obs)

    # compute the first coefficient of a linear regression model at a given station
    def coef(data_group):
        x = data_group[["Year"]]
        y = data_group["Temp"]
        LR = LinearRegression()
        LR.fit(x, y)
        return LR.coef_[0]
    
    coefs = df.groupby(["NAME", "LATITUDE", "LONGITUDE"]).apply(coef).reset_index(name = "temp_diff")

    month

Now let's take a look at the January climate data in India from 1980 to 2020. What will it be like when we visualize it using the function we have written?

In [30]:
# assumes you have imported necessary packages
color_map = px.colors.diverging.RdGy_r # choose a colormap

fig = temperature_coefficient_plot("climate.db", "India", 1980, 2020, 1, 
                                   min_obs = 10,
                                   zoom = 2,
                                   mapbox_style="carto-positron",
                                   color_continuous_scale=color_map)

fig.show()

Let's use the same function on another time frame for another country. For example, let's look at the temperature change in Germany from the year 2000 to 2020 in March. What will our map be like?

In [32]:
color_map = px.colors.diverging.Tropic # choose a colormap

fig = temperature_coefficient_plot("climate.db", "Germany", 2000, 2020, 3, 
                                   min_obs = 20,
                                   zoom = 4,
                                   mapbox_style="carto-positron",
                                   color_continuous_scale=color_map)

fig.show()

## Create Two More Interesting Figures
Now that we have created a geographic visualization of the dataset, let's go ahead and create two more interesting figures from the data.

Let's explore the relationship between station elevation and temperature variability. Our analysis will focus on understanding how stations at different elevations experience temperature changes throughout the year, which could provide valuable insights into local climate patterns.

Let's create another querying function called `query_station_temperature_data` to retrieve data from the climate database at once. This function retrieves station-level temperature data for a given country and time, and only selects stations with more than 3 readings in the same month across years.

In [35]:
from climate_database import query_station_temperature_data
print(inspect.getsource(query_station_temperature_data))

def query_station_temperature_data(db_file, country, year_begin, year_end, month):
    """
    input:
    db_file: the file name for the database.
    country: a string giving the name of a country.
    year_begin, year_end: start and end year (inclusive)
    month: specific month.

    output: a dataframe with the following columns
    NAME: name of the station
    LATITUDE, LONGITUDE: location of the station
    STNELEV: elevation of the station
    Country: the name of the country in which the station is located.
    Year: the Year for the reading
    Month: the Month for the reading
    Temp_Range: range of temperature in the station
    Num_Readings: number of readings for the station
    """

    # connect to database
    conn = sqlite3.connect(db_file)


    # write sql command using f-strings
    # only select stations with more than 3 readings in a same month across years
    command = f"""
    WITH yearly_station_counts AS (
        SELECT S.NAME, COUNT(DISTINCT T.Year) as ye

Let's check the output of this query function using the same example we have before.

In [37]:
query_station_temperature_data(db_file = "climate.db",
                               country = "India", 
                               year_begin = 1980,
                               year_end = 2000,
                               month = 1)

,NAME,LATITUDE,LONGITUDE,STNELEV,Country,Month,Temp_Range
0,AGARTALA,23.8830,91.2500,16.0,India,1,2.95
1,AGRA,27.1667,78.0333,168.0,India,1,2.20
2,AHMADABAD,23.0670,72.6330,55.0,India,1,3.54
3,AKOLA,20.7000,77.0330,282.0,India,1,4.60
4,AKOLA,20.7000,77.0670,305.0,India,1,3.63
...,...,...,...,...,...,...,...
92,TIRUCHCHIRAPALLI,10.7670,78.7170,88.0,India,1,2.79
93,TRIVANDRUM,8.5000,77.0000,64.0,India,1,2.10
94,VARANASI_BABATPUR,25.4500,82.8670,85.0,India,1,0.93
95,VERAVAL,20.9000,70.3670,8.0,India,1,3.47


With the help of the querying function, we can now answer the first question: How does temperature variability differ between high-elevation and low-elevation stations in a country in a given month? 

We will create an interactive box plot. The X-axis will be the elevation bins, whereas the Y-axis will represent temperature range per station.

In [39]:
from climate_database import plot_elevation_temp_variability
print(inspect.getsource(plot_elevation_temp_variability))

def plot_elevation_temp_variability(db_file, country, year_begin, year_end, month, num_bins=5):

    """
    input:
    db_file: database file path
    country: country name
    year_begin, year_end: time period to analyze
    month: month to analyze (1-12)
    num_bins: number of elevation bins to create
    """
    # Get the data
    df = query_station_temperature_data(db_file, country, year_begin, year_end, month)
    
    # Create elevation bins
    df['Elevation_Bin'] = pd.qcut(df['STNELEV'], 
                                q=num_bins, 
                                labels=[f'{int(x.left)}-{int(x.right)}m' 
                                       for x in pd.qcut(df['STNELEV'], q=num_bins).unique()])
    
    # Create box plot
    fig = px.box(
        df,
        x='Elevation_Bin',
        y='Temp_Range',
        points='all',  # Show all points
        hover_data=['NAME', 'STNELEV'],  # Show station details on hover
        labels={
            'Elevation_Bin': 'Elevation Rang

Let's generate the graph:

In [41]:
fig = plot_elevation_temp_variability("climate.db", "India", 1980, 2020, 1, num_bins=5)
fig.show()

Great. Now let's answer our second question: How does the correlation between elevation and temperature variability change over time?

We will created a faceted scatter plot, where the x-axis is the elvation, and y-axis represents the temperature range. We will also add a trendline showing linear regression per decade.

In [43]:
from climate_database import plot_elevation_temp_trends
print(inspect.getsource(plot_elevation_temp_trends))

def plot_elevation_temp_trends(db_file, country, decade_ranges, month, max_elevation = 9000):
    """
    input:
    db_file: database file path
    country: country name
    decade_ranges: list of tuples, each containing (start_year, end_year) for a decade
    month: month to analyze (1-12)
    """

    # Initialize list to store data from each decade
    decade_data = []
    
    # Collect data for each decade
    for start_year, end_year in decade_ranges:
        df = query_station_temperature_data(db_file, country, start_year, end_year, month)
        # Filter out stations with unrealistic elevations
        df = df[df['STNELEV'] <= max_elevation]
        df['Decade'] = f'{start_year}s'
        decade_data.append(df)
    
    # Combine all decades' data
    combined_df = pd.concat(decade_data, ignore_index=True)
    
    # Create the title string including the max elevation
    title_string = f'Evolution of Elevation-Temperature Relationship in {country}<br>Month {month} (Stations 

In [44]:
fig = plot_elevation_temp_trends("climate.db", 
                                 "Germany", 
                                 ((1980, 1989),(1990, 1999), (2000, 2009), (2010, 2019)), 
                                 3,
                                max_elevation = 9000)
fig.show()

From the graph, we see that the correlation between elevation and temperature varaibility is not so stable across four decades in Germany in March. 

Great! Now we have learned to create visualizations using database.